In [ ]:
"""
En este paso filtramos el dataset de alumnos por el año 2016 y hacemos join con el dataset de escuelas.
Luego eliminamos NULLs y grabamos el file df_final.csv. Pensando en predecir el campo nivel_desemp_matematica, 
lo transofrmamos en dos categorias, SATIFACTORIO con valor 0 (satifactorio y avanzado) y 1 BÁSICO (básico y por debajo de básico).
Para ejecutar este paso deben tener en la carpeta /desafio3/datasets , los datasets app_alumno.csv y app_escuela.csv.
"""

import pandas as pd
df = pd.read_csv('../desafio3/datasets/app_alumno.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_escuelas = pd.read_csv('../desafio3/datasets/app_escuela.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_provincia = pd.read_csv('../desafio3/datasets/app_provincia.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_departamento = pd.read_csv('../desafio3/datasets/app_departamento.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_salario = pd.read_csv('../desafio3/datasets/app_salario.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_gastonivel = pd.read_csv('../desafio3/datasets/app_gastonivel.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_gastoobjeto = pd.read_csv('../desafio3/datasets/app_gastoobjeto.csv', parse_dates=True, na_values=['nc'], low_memory=False)


In [2]:
#Nos quedamos con los casos sin nulos en desmpeño de matemática.
#También nos quedamos con el año 2016 y los exámenes de primaria
df = df[(df['nivel_desemp_matematica'].notnull()) & (df['year_id'] == 2016) & (df['nivel_id'] == 2)]

In [3]:
#Identificamos como avanzado y satisfactorio el desmpeño en matematica segun la nota
df['nivel_desemp_matematica'] = df['nivel_desemp_matematica'].apply(lambda x: 0 if x in ['satisfactorio','avanzado'] else 1)

In [4]:
#Eliminamos columnas que no vamos a usar en el modelo y aquellas observaciones que tengan na
df = df.drop(['id','repeticion_secundaria','nivel_desemp_ciencias_sociales','nivel_desemp_ciencias_naturales','year_id', 'nivel_id'], axis=1)
df = df.dropna(axis=0)

In [5]:
#### SE AGREGA JOIN PARA AGREGAR AL DATAFRAME LA CANTIDAD DE ALUMNOS QUE TIENE CADA ESCUELA
df_curso = pd.read_csv('../desafio3/datasets/app_curso.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df_curso = df_curso[df_curso['year_id'] == 2016]
df_curso_gr = df_curso.groupby(by='escuela_id')['matricula'].sum()
df_curso_gr = df_curso_gr.reset_index()
df_escuelas = df_escuelas.rename(columns={'id': 'escuela_id'})
df_escuelas = df_escuelas.merge(df_curso_gr, how='inner', on ='escuela_id')
####

In [6]:
df = df.merge(df_escuelas, how='inner', on ='escuela_id')
df = df.drop('icse_cat', axis=1)
df = df.dropna(axis=0)

In [7]:
df_curso_prov = df_curso.merge(df_escuelas, how = 'left', on ='escuela_id')

In [8]:
df_curso_prov = df_curso_prov.groupby(by='provincia_id')['matricula_x'].sum()

In [9]:
df_curso_prov = pd.DataFrame(df_curso_prov)
df_curso_prov['id'] = df_curso_prov.index


In [10]:
# Limpieza del dataset de salario
df_salario = df_salario[(df_salario['year_id'] == 2016)]
df_salario = df_salario.groupby(['provincia_id', 'name'])['value'].aggregate('mean').unstack()


In [11]:
# Limpieza del dataset de gasto nivel
df_gastonivel = df_gastonivel[(df_gastonivel['year_id'] == 2016)]
df_gastonivel = df_gastonivel.groupby(['provincia_id', 'name'])['value'].aggregate('mean').unstack()
df_gastonivel = df_gastonivel.merge(df_curso_prov, how='inner', left_on=['provincia_id'],right_on=['id'])
df_gastonivel = df_gastonivel[['inicial+primaria', 'privada', 'secundaria', 'sin_discriminar', 'snu']].div(df_gastonivel.matricula_x, axis=0)
df_gastonivel['provincia_id'] = df_gastonivel.index

In [12]:
# Limpieza del dataset de gasto objeto
df_gastoobjeto = df_gastoobjeto[(df_gastoobjeto['year_id'] == 2016)]
df_gastoobjeto = df_gastoobjeto.groupby(['provincia_id', 'name'])['value'].aggregate('mean').unstack()
df_gastoobjeto = df_gastoobjeto.merge(df_curso_prov, how='inner', left_on=['provincia_id'],right_on=['id'])
df_gastoobjeto = df_gastoobjeto[['bienes_y_servicios_no_personales', 'erogacion_de_capital', 'personal', 'transferencias_educacion_privada', 'transferencias_otras']].div(df_gastoobjeto.matricula_x, axis=0)
df_gastoobjeto['provincia_id'] = df_gastoobjeto.index

In [13]:
# Merge de los dataset limpios

df = df.merge(df_salario, how = 'inner', on = 'provincia_id')
df = df.merge(df_gastonivel, how = 'inner', on = 'provincia_id')
df = df.merge(df_gastoobjeto, how = 'inner', on = 'provincia_id')

In [14]:
df.head()

,sexo,indice_socioeconomico,nivel_desemp_matematica,nivel_desemp_lengua,ponderador_lengua,ponderador_matematica,ponderador_ciencias_naturales,ponderador_ciencias_sociales,tiene_notebook,tiene_pc,...,inicial+primaria,privada,secundaria,sin_discriminar,snu,bienes_y_servicios_no_personales,erogacion_de_capital,personal,transferencias_educacion_privada,transferencias_otras
0,f,medio,0,avanzado,1.309467,1.308268,-1.0,-1.0,no,si,...,14153.324289,6424.569041,9509.173296,5318.844228,2564.942087,3060.587468,1537.060193,25583.003705,6424.569041,1365.632533
1,f,alto,0,avanzado,1.309467,1.308268,-1.0,-1.0,si,si,...,14153.324289,6424.569041,9509.173296,5318.844228,2564.942087,3060.587468,1537.060193,25583.003705,6424.569041,1365.632533
2,m,medio,0,satisfactorio,1.309467,1.308268,-1.0,-1.0,si,si,...,14153.324289,6424.569041,9509.173296,5318.844228,2564.942087,3060.587468,1537.060193,25583.003705,6424.569041,1365.632533
3,m,medio,0,avanzado,1.309467,1.308268,-1.0,-1.0,no,si,...,14153.324289,6424.569041,9509.173296,5318.844228,2564.942087,3060.587468,1537.060193,25583.003705,6424.569041,1365.632533
4,m,medio,0,avanzado,1.309467,1.308268,-1.0,-1.0,si,si,...,14153.324289,6424.569041,9509.173296,5318.844228,2564.942087,3060.587468,1537.060193,25583.003705,6424.569041,1365.632533


In [15]:
df.shape

(277573, 47)

In [ ]:
print('#######SHAPE:','\n',df.shape,'\n','#######','\n')
print('#######VALUE_COUNTS:','\n',df['nivel_desemp_matematica'].value_counts(),'\n','#######','\n')
print('#######NULLs:','\n',(df.isnull().sum()/df.shape[0])*100,'\n','#######','\n')
print('#######BALANCE:','\n',df['nivel_desemp_matematica'].value_counts(),'\n','#######','\n')
df.to_csv('../desafio3/datasets/df_final.csv')

In [ ]:
df.columns

In [ ]:
"""
En este paso, levantamos el nuevo dataset y convertimos al tipo de dato category las columnas que consideramos categoricas
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

df = pd.read_csv('../desafio3/datasets/df_final.csv', parse_dates=True, na_values=['nc'], low_memory=False)
df = df.drop('Unnamed: 0', axis=1)
for i in df.columns:
    if (df[i].dtype == object) or i in ['nivel_id', 'escuela_id', 'provincia_id', 'departamento_id']:
        df[i] = pd.Categorical(df[i])



In [ ]:

"""A continuación ejecutamos modelos de regresión logistica para validar que el dataset final sea adecuado para trabajar .
"""
X = df.drop('nivel_desemp_matematica', axis=1)
y = df['nivel_desemp_matematica']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)


In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns = 'n'):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            #c para tomar columnas categoricas, cualquier otro valor para tomar continuas.
            if self.columns == 'c':
                categorical_columns = [col for col in X.columns if pd.api.types.is_categorical_dtype(df[col])]
                return pd.DataFrame(X[categorical_columns])
            else:
                scal_columns = [col for col in X.columns if (pd.api.types.is_categorical_dtype(df[col]) == False)]
                return pd.DataFrame(X[scal_columns])
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self
cat_pipe = make_pipeline(ColumnSelector('c'),OneHotEncoder(handle_unknown='ignore'))
num_pipe = make_pipeline(ColumnSelector('n'),StandardScaler())
union = make_union(num_pipe,
                   cat_pipe)



In [ ]:
#Este paso tarda bastante dado que ejecutamos un random grid search.
"""
RESULTADO: 
Best score: 0.739
Best parameters set:
	 log__C: 0.1
	 log__penalty: 'l2'
              precision    recall  f1-score   support

           0       0.77      0.81      0.79     44028
           1       0.69      0.63      0.66     29395

    accuracy                           0.74     73423
   macro avg       0.73      0.72      0.72     73423
weighted avg       0.73      0.74      0.74     73423

0.7373302643585797
"""
union.fit_transform(X)

pipeline = Pipeline([('union', union), 
                     ('log', LogisticRegression(solver='liblinear'))])

parameters = {'log__penalty': ['l1', 'l2'],
              'log__C': [0.01, 0.1, 1, 10, 100]}
folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
grid_search = RandomizedSearchCV(pipeline, parameters,cv=folds, n_jobs = 5)
grid_search.fit(pd.DataFrame(X_train, columns=X.columns), y_train)

print("Best score: %0.3f" % grid_search.best_score_) 
print("Best parameters set:" )
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted (parameters . keys()): 
                    print("\t %s: %r" % (param_name, best_parameters[param_name])) 
y_pred = grid_search.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))